In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing

2023-08-21 01:43:27.432677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 01:43:28.033042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


In [2]:
#limit VRAM usage
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2023-08-21 01:43:28.618444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 01:43:28.652335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 01:43:28.652500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
#Load the dataset and create a dataframe with image paths and labels
dataset_path = "datasets/dataset2celebA/Train/"
images = []
labels = []
for folder in os.listdir(dataset_path):
    if os.path.isdir(os.path.join(dataset_path, folder)):
        for file in os.listdir(os.path.join(dataset_path, folder)):
            if file.endswith(".jpg"):
                images.append(os.path.join(dataset_path, folder, file))
                labels.append(folder)
df = pd.DataFrame({"image": images, "label": labels})

In [4]:
#Load the validation set and create a dataframe with image paths and labels
validation_path = "datasets/dataset2celebA/Validation/"
images = []
labels = []
for folder in os.listdir(validation_path):
    if os.path.isdir(os.path.join(validation_path, folder)):
        for file in os.listdir(os.path.join(validation_path, folder)):
            if file.endswith(".jpg"):
                images.append(os.path.join(validation_path, folder, file))
                labels.append(folder)
vf = pd.DataFrame({"image": images, "label": labels})

In [5]:
#Define a function to preprocess the images: resize, grayscale, histogram equalization
def preprocess_image(image_path):
  image = cv2.imread(image_path)
  image = cv2.resize(image, (64, 64))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = cv2.equalizeHist(image)
  image = image / 255.0
  image = np.expand_dims(image, axis=2)
  return image

In [6]:
#Apply the preprocessing function to the images and convert the labels to numeric values
X_train = np.array([preprocess_image(image) for image in df["image"]])
y_train = np.array([0 if label == "male" else 1 for label in df["label"]])

In [7]:
X_test = np.array([preprocess_image(image) for image in vf["image"]])
y_test = np.array([0 if label == "male" else 1 for label in vf["label"]])
#label for male is 0 & female is 1
#1 is privilage group & 0 is underprivilege group

In [8]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation="relu", input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

' model = Sequential()\n\nmodel.add(Conv2D(64, (3, 3), activation="relu", input_shape=(64, 64, 1)))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Conv2D(128, (3, 3), activation="relu"))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Conv2D(256, (3, 3), activation="relu"))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Conv2D(512, (3, 3), activation="relu"))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Flatten())\n\nmodel.add(Dense(1024, activation="relu"))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(512, activation="relu"))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(256, activation="relu"))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(1, activation="sigmoid"))\nmodel.summary() '

In [14]:
""" from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

# Freeze the layers in the pre-trained model
for layer in vgg16.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
model = Sequential()
model.add(Conv2D(3, (3, 3), activation="relu", input_shape=(64, 64, 1)))
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary() """

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 3)         30        
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 14,846,303
Trainable params: 131,615
Non

In [ ]:
#Compile and fit the model on the train set
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping = EarlyStopping(monitor="val_loss", patience=5)
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [15]:
""" from tensorflow.keras.optimizers import Adam

# Compile the model with Adam optimizer and binary crossentropy loss
model.compile(optimizer=Adam(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Set up early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

# Train the model with the new data and callbacks
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping]) """

2023-08-21 02:08:50.945151: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2621440000 exceeds 10% of free system memory.
2023-08-21 02:09:01.671783: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.44GiB (rounded to 2621440000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-21 02:09:01.671810: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-21 02:09:01.671817: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 42, Chunks in use: 42. 10.5KiB allocated for chunks. 10.5KiB in use in bin. 1.3KiB client-requested in use in bin.
2023-08-21 02:09:01.671822: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 4, Chunks in use: 4. 2.0K

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

01.672276: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f8795897400 of size 285952 next 75
2023-08-21 02:09:01.672279: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f87958dd100 of size 442368 next 77
2023-08-21 02:09:01.672282: I tensorflow/tsl/framework/bfc_allocator.cc:1095] Free  at 7f8795949100 of size 1048576 next 105
2023-08-21 02:09:01.672285: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f8795a49100 of size 1015808 next 80
2023-08-21 02:09:01.672289: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f8795b41100 of size 782080 next 18446744073709551615
2023-08-21 02:09:01.672292: I tensorflow/tsl/framework/bfc_allocator.cc:1100]      Summary of in-use Chunks by size: 
2023-08-21 02:09:01.672296: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 42 Chunks of size 256 totalling 10.5KiB
2023-08-21 02:09:01.672300: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 4 Chunks of size 512 totalling 2.0KiB
2023-08-21 02:09:01.672304: I

In [ ]:
#Plot the training and validation accuracy and loss curves
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train accuracy")
plt.plot(history.history["val_accuracy"], label="validation accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#Evaluate the model on the test set
y_pred = model.predict(X_test).round().ravel()
test_acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
test_cm = confusion_matrix(y_test, y_pred)
print("Test accuracy:", test_acc)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test confusion matrix:")
print(test_cm)

In [ ]:
#Create a binary label dataset from the train set for aif360
train_df = pd.DataFrame(X_train.reshape(-1, 64*64))
train_df["label"] = y_train
train_df["gender"] = 0
train_dataset = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=train_df, label_names=["label"], protected_attribute_names=["gender"], unprivileged_protected_attributes=[0])

In [ ]:
print(train_dataset.features.shape)
print(train_dataset.features.size)

#Extract the features and labels from the reweighted train set
X_train = train_dataset.features[:, :-1].reshape(-1, 64, 64, 1)
y_train = train_dataset.labels.ravel()

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric
#Compute the disparate impact and statistical parity difference metrics for the train set
metric_dataset = BinaryLabelDatasetMetric(train_dataset, unprivileged_groups=[{"gender": 0}], privileged_groups=[{"gender": 1}])
print("Disparate impact:", metric_dataset.disparate_impact())
print("Statistical parity difference:", metric_dataset.statistical_parity_difference())

In [ ]:
#Apply the reweighing algorithm to mitigate bias in the training set
RW = Reweighing(unprivileged_groups=[{"gender": 0}], privileged_groups=[{"gender": 1}])
train_dataset_rw = RW.fit_transform(train_dataset)

#Compute the metrics for the reweighted train set
metric_dataset_rw = BinaryLabelDatasetMetric(train_dataset_rw, unprivileged_groups=[{"gender": 0}], privileged_groups=[{"gender": 1}])
print("Disparate impact after reweighing:", metric_dataset_rw.disparate_impact())
print("Statistical parity difference after reweighing:", metric_dataset_rw.statistical_parity_difference())

In [ ]:
print(train_dataset_rw.features.shape)
print(train_dataset_rw.features.size)

In [ ]:
#Extract the features and labels from the reweighted train set
X_train_rw = train_dataset_rw.features[:, :-1].reshape(-1, 64, 64, 1)
y_train_rw = train_dataset_rw.labels.ravel()

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

In [ ]:
#Compile and fit the model on the reweighted train set
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping = EarlyStopping(monitor="val_loss", patience=5)
history = model.fit(X_train_rw, y_train_rw, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
#Plot the training and validation accuracy and loss curves
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train accuracy")
plt.plot(history.history["val_accuracy"], label="validation accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#Evaluate the model on the test set
y_pred = model.predict(X_test).round().ravel()
test_acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
test_cm = confusion_matrix(y_test, y_pred)
print("Test accuracy:", test_acc)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test confusion matrix:")
print(test_cm)

In [ ]:
#Create a binary label dataset from the test set for aif360
test_df = pd.DataFrame(X_test.reshape(-1, 64*64))
test_df["label"] = y_test
test_df["gender"] = 0
test_dataset = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=test_df, label_names=["label"], protected_attribute_names=["gender"], unprivileged_protected_attributes=[0])

In [ ]:
# Compute the classification metrics for the test set
metric_classifier = ClassificationMetric(test_dataset, test_dataset.copy(), unprivileged_groups=[{"gender": 0}], privileged_groups=[{"gender": 1}])
print("Accuracy:", metric_classifier.accuracy())
#print("Balanced accuracy:", metric_classifier.balanced_accuracy())
print("Equal opportunity difference:", metric_classifier.equal_opportunity_difference())
print("Average odds difference:", metric_classifier.average_odds_difference())
print("Theil index:", metric_classifier.theil_index())